In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from requests import get
import time
import pandas as pd
from random import randint
import datetime

In [13]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import re

In [3]:
# import json as df
df = pd.read_json ('cs4300sp2020-jjl296-ec629-cc2459-tb444-ak778/data/total_data_2.json', convert_dates=False)

In [12]:
df["transcript"][0]

'16-year-old TJ Kim may not have his driver’s license yet, but he has been flying planes all over Virginia in order to deliver medical supplies to rural hospitals fighting the novel coronavirus.\n\nThe teen, who is a sophomore at Landon School in Bethesda, Maryland, was inspired to embark on his delivery missions after his classes were canceled as a result of COVID-19.\n\nWithout school or lacrosse to occupy his time, he turned his focus on his flying lessons.\n\nRELATED: Man Uses His Savings to Fill Up Gas Tanks For Dozens of Nurses Heading to Nearby Hospital\n\nAccording to The Associated Press, Kim and his family then launched their Operation SOS (Supplies Over Skies) project as a means of keeping busy during the shutdowns while simultaneously serving others in need by delivering supplies to rural hospitals.\n\nWhen Kim first asked his flight instructor, Dave Powell, to use their flying lessons as delivery runs for Virginia hospitals, the pilot was awestruck.\n\nLOOK: College Studen

In [18]:
def tokenize(text, stem_words = True):
    """Returns a tuple containing 
        a list of words that make up the text
        and the number of total words (not including stop words).
    
    Note: for simplicity, lowercase everything.
    Requirement: Use Regex to satisfy this function
    
    Arguments
    =========
    text: String
    stem_words: boolean
    
    Returns
    =======
    Tuple of a list and number 
    
    """
    stop_words = stopwords.words('english')
    stemmer = PorterStemmer()
    regex = r"([a-zA-Z]+)"
    tokens = re.findall(regex, text.lower())
    
    numwords = 0
    words = []
    for t in tokens:
        if not t in stop_words:
            numwords += 1
            if (stem_words):
                words.append(stemmer.stem(t))
            else:
                words.append(t)
    return (words, numwords)

In [19]:
tokenize(df["transcript"][0])

(['year',
  'old',
  'tj',
  'kim',
  'may',
  'driver',
  'licens',
  'yet',
  'fli',
  'plane',
  'virginia',
  'order',
  'deliv',
  'medic',
  'suppli',
  'rural',
  'hospit',
  'fight',
  'novel',
  'coronaviru',
  'teen',
  'sophomor',
  'landon',
  'school',
  'bethesda',
  'maryland',
  'inspir',
  'embark',
  'deliveri',
  'mission',
  'class',
  'cancel',
  'result',
  'covid',
  'without',
  'school',
  'lacross',
  'occupi',
  'time',
  'turn',
  'focu',
  'fli',
  'lesson',
  'relat',
  'man',
  'use',
  'save',
  'fill',
  'ga',
  'tank',
  'dozen',
  'nurs',
  'head',
  'nearbi',
  'hospit',
  'accord',
  'associ',
  'press',
  'kim',
  'famili',
  'launch',
  'oper',
  'so',
  'suppli',
  'sky',
  'project',
  'mean',
  'keep',
  'busi',
  'shutdown',
  'simultan',
  'serv',
  'other',
  'need',
  'deliv',
  'suppli',
  'rural',
  'hospit',
  'kim',
  'first',
  'ask',
  'flight',
  'instructor',
  'dave',
  'powel',
  'use',
  'fli',
  'lesson',
  'deliveri',
  'run',


In [57]:
def make_countsdict(tokens):
    worddict = {}
    for word in tokens:
        if word in worddict:
            worddict[word] += 1
        else:
            worddict[word] = 1
    return worddict

In [80]:
def make_keywords(row, title_scale = 0.3):
    tokenized = tokenize(row["transcript"])
    words = tokenized[0]
    numwords = tokenized[1]
    worddict = make_countsdict(words)
    # divide by numwords
    counts = [(word,count) for word,count in worddict.items()]
    freq = [(word, count/numwords) for word, count in counts]

    # tokenize title
    tokenized_title = tokenize(row["title"])
    titledict = make_countsdict(tokenized_title[0])
    title_freq = [(word, count/tokenized_title[1]) for word, count in titledict.items()]

    # combine
    title_scaled = [(word, count*title_scale) for word, count in title_freq]

    # get top 20
    total = freq + title_scaled
    sortedtotal = sorted(total, key=lambda x: x[1], reverse=True) 
    top_20 = {}
    top = 0
    for word,freq in sortedtotal:
        if word not in top_20:
            top_20[word] = freq
            top += 1
            if top == 20: 
                break
    row["keywords"] = top_20

In [81]:
for index, row in df.iterrows():
    print(index)
    make_keywords(row)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [76]:
df["keywords"] = ""

In [82]:
df

,title,url,date,transcript,source,keywords
0,16-Year-Old Has Been Using His Flying Lessons ...,https://www.goodnewsnetwork.org/teen-flies-sup...,4/9/20,16-year-old TJ Kim may not have his driver’s l...,goodnewsnetwork.org,"{'hospit': 0.034482758620689655, 'year': 0.024..."
1,Tyler Perry Picks Up the Tab for All Groceries...,https://www.goodnewsnetwork.org/tyler-perry-pa...,4/9/20,It’s not uncommon for movie mogul Tyler Perry ...,goodnewsnetwork.org,"{'perri': 0.038461538461538464, 'tyler': 0.032..."
2,Taking Hot Baths Every Day is Linked to Lower ...,https://www.goodnewsnetwork.org/daily-hot-bath...,4/8/20,Regular tub bathing is linked to a lower risk ...,goodnewsnetwork.org,"{'bath': 0.04896907216494845, 'risk': 0.028350..."
3,Watch Farmer Use His Tractor to Plow Massive T...,https://www.goodnewsnetwork.org/farmer-plows-t...,4/8/20,A talented farmer has created his own heartwar...,goodnewsnetwork.org,"{'watch': 0.03, 'farmer': 0.03, 'use': 0.03, '..."
4,3 Ways to Deal With the Anxiety of a New Situa...,https://www.goodnewsnetwork.org/dr-brene-brown...,4/8/20,The Lesson: If you have been stressing out ami...,goodnewsnetwork.org,"{'way': 0.03, 'deal': 0.03, 'anxieti': 0.03, '..."
...,...,...,...,...,...,...
1057,How has your community helped resolve a local ...,https://www.theguardian.com/uk-news/2017/dec/2...,2/12/18,When a small town in Devon banned plastic bags...,The Guardian,"{'commun': 0.04285714285714285, 'help': 0.0428..."
1058,The Upside – sign up here for a weekly digest ...,https://www.theguardian.com/world/2018/feb/12/...,2/12/18,The Guardian is launching a new strand of jour...,The Guardian,"{'guardian': 0.034482758620689655, 'upsid': 0...."
1059,About 'the Upside' - a Guardian series,https://www.theguardian.com/info/2018/feb/12/a...,2/12/18,The Upside series comprises journalism that fo...,The Guardian,"{'upsid': 0.09999999999999999, 'guardian': 0.0..."
1060,10 great Finnish innovations,https://www.theguardian.com/world/2018/feb/12/...,2/12/18,Maternity pack Finland’s “baby box” was introd...,The Guardian,"{'great': 0.09999999999999999, 'finnish': 0.09..."


In [9]:
df = df.drop(df.columns[0], axis = 1)

In [83]:
df.to_json('total_data_2_keywords.json', orient='records')

In [84]:
with open('total_data_2_keywords.json', 'w', encoding='utf-8') as file:
    df.to_json(file, force_ascii=False, orient='records')